In [2]:
from fco2models.models import UNet2DModelWrapper
from fco2models.ueval import load_models
model_info = {
    #'seas_noattn_100': ['../models/renko/seasonal/noattn/', 'e_100.pt', UNet2DModelWrapper],
    #'noattn_100': ['../models/renko/unet2d_noattn/', 'e_100.pt', UNet2DModelWrapper],
    #'conv_attn_220': ['../models/renko/unet2d_fixsplit/', 'e_220.pt', UNet2DModelWrapper],
    'pp_100': ['../models/constlr/pointpos/', 'e_100.pt', UNet2DModelWrapper],
    #'attn_first_50': ['../models/renko/attn_first/', 'e_50.pt', UNet2DModelWrapper],
    #'conv_attn_300': ['../models/renko/unet2d_fixsplit/', 'e_300.pt', UNet2DModelWrapper]
}
models = load_models(model_info)

In [3]:
# compute these values for all models
from fco2models.ueval import print_loss_info
for key in models.keys():
    print(f'loading {key}')
    print_loss_info(models[key])

loading pp_100
min epoch: 97
min train loss: 0.017852500817962372
min val loss epoch: 98
min val loss: 0.07272534998690088
----------------------------------


In [4]:
import pandas as pd
import numpy as np
from fco2models.utraining import prep_df, make_monthly_split, get_segments_random, get_segments, get_context_mask, normalize_dss, get_stats_df
DATA_PATH = "../data/training_data/"
df = pd.read_parquet(DATA_PATH + "SOCAT_1982_2021_grouped_colloc_augm_bin.pq", engine='pyarrow')[:10000]
df = prep_df(df, bound=True)[0]
#map expocode column to int
expocode_map = df['expocode'].unique()
expocode_map = {expocode: i for i, expocode in enumerate(expocode_map)}
df['expocode_id'] = df['expocode'].map(expocode_map) 
print(df.columns)

2025-06-02 11:05:39,097 - INFO - salinity stacking
2025-06-02 11:05:39,106 - INFO - adding positional and temporal encodings
2025-06-02 11:05:39,124 - INFO - add climatology data
2025-06-02 11:05:48,007 - INFO - adding xco2 data
2025-06-02 11:05:48,300 - INFO - adding seamask data
2025-06-02 11:14:39,891 - INFO - removing xco2 levels from fco2rec_uatm
2025-06-02 11:14:39,896 - INFO - replacing outliers with Nans, fco2rec_uatm > 400


Index(['index', 'expocode', 'window_id', 'time_1d', 'lat', 'lon', 'sal',
       'sst_deg_c', 'pppp_hpa', 'woa_sss', 'ncep_slp_hpa', 'dist_to_land_km',
       'fco2rec_uatm', 'fco2rec_flag', 'time', 'year', 'temp_soda',
       'salt_soda', 'mld_dens_soda', 'lon_soda', 'lat_soda', 'depth_soda',
       'time_soda', 'sst_cci', 'sst_cci_uncertainty', 'ice_cci', 'time_sstcci',
       'lat_sstcci', 'lon_sstcci', 'interpolated', 'bin_id', 'segment_id',
       'ssh_adt', 'ssh_sla', 'lat_sshcm', 'lon_sshcm', 'time_sshcm',
       'chl_globcolour', 'chl_globcolour_uncert', 'chl_globcolour_flags',
       'time_chlgc', 'lat_chlgc', 'lon_chlgc', 'sss_cci',
       'sss_cci_random_error', 'lat_ssscci', 'lon_ssscci', 'time_ssscci',
       'day_of_year', 'sin_day_of_year', 'cos_day_of_year', 'sin_lat',
       'sin_lon_cos_lat', 'cos_lon_cos_lat', 'sin_lon', 'cos_lon', 'is_north',
       'co2_clim8d', 'xco2', 'seamask', 'expocode_id'],
      dtype='object')


In [5]:
expocode_map

{'069920180814': 0,
 '069920180921': 1,
 '069920181126': 2,
 '069920190503': 3,
 '069920190509': 4,
 '069920190602': 5,
 '069920190803': 6,
 '069920200703': 7,
 '069920201009': 8,
 '069920201108': 9,
 '069920201125': 10,
 '069920201211': 11,
 '069920201226': 12,
 '069920210106': 13}